In [1]:
# IMPORTS
import pandas as pd

from enron.src.common.const import *

from manufacturing_company.src.common.const import *
from manufacturing_company.src.features.employee_activity import *
from manufacturing_company.src.features.neighborhood_variability import *


In [2]:
# CREATE FEATURES
for month in range(MONTHS):
    df = pd.read_csv(FILE_MINIMUM_ACTIVITY.format(month), sep=';', parse_dates=[EVENT_DATE])
    df_overtime = calculate_overtime(df, from_hour=17, to_hour=5)
    df_weekend = calculate_work_at_weekend(df)
    df_merged = pd.merge(df_overtime, df_weekend, on=SENDER)
    
    df_network = pd.read_csv(FILE_NETWORK_MEASURES.format(month), sep=';')
    df_merged = pd.merge(df_network, df_merged, left_on=ID, right_on=SENDER)
    
    sender, recipient, sender_recipient = calculate_neighborhood_variability(df, df_merged[ID])
    
    sender = pd.DataFrame.from_dict(sender, orient='index', columns=[NEIGHBORHOOD_VARIABILITY_SENDER])
    sender.index.name = ID
    
    recipient = pd.DataFrame.from_dict(recipient, orient='index', columns=[NEIGHBORHOOD_VARIABILITY_RECIPIENT])
    recipient.index.name = ID
    
    sender_recipient = pd.DataFrame.from_dict(sender_recipient, orient='index', columns=[NEIGHBORHOOD_VARIABILITY_ALL])
    sender_recipient.index.name = ID
    
    df_merged = pd.merge(df_merged, sender, on=ID)
    df_merged = pd.merge(df_merged, recipient, on=ID)
    df_merged = pd.merge(df_merged, sender_recipient, on=ID)
    
    df_merged.to_csv(FILE_FEATURES.format(month), sep=';', index=False)
